In [37]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np
from datetime import date, datetime
from processing import Process

conf = SparkConf()
# conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.0')
# conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')
spark = SparkSession.builder.config(conf=conf).getOrCreate()

procdata = Process()

In [ ]:
# CONVERT FILES FROM CSV TO JSON AND XML

geralprounicsv = pd.read_csv("pure_data/GERAL-cursos-prouni.csv")
addprounicsv = pd.read_csv("pure_data/pda-prouni.csv", sep=';')

geralprounicsv.to_xml("pure_data/geralprouni.xml")
addprounicsv.to_json("pure_data/prouni.json")

In [78]:
geralprouni = pd.DataFrame(pd.read_xml("pure_data/geralprouni.xml"))

quantidade_bolsas = geralprouni[['bolsa_integral_cotas', 'bolsa_integral_ampla', 'bolsa_parcial_cotas', 'bolsa_parcial_ampla']].sum(axis=1)
nota_bolsa = geralprouni[['nota_integral_ampla', 'nota_integral_cotas', 'nota_parcial_ampla', 'nota_parcial_cotas']].mean(axis=1)

prouni = pd.DataFrame({
    'mensalidade': geralprouni['mensalidade'], 
    'estado': geralprouni['uf_busca'], 
    'curso': geralprouni['nome'], 
    'quantidade_bolsas': quantidade_bolsas,
    'nota_bolsa': nota_bolsa
})
#geralprouni.head(5)
print(prouni.__len__(), "rows")
prouni.sample(n=10)

41447 rows


,mensalidade,estado,curso,quantidade_bolsas,nota_bolsa
38393,237.00,PR,História,2.0,558.990
2335,1440.86,RO,Engenharia Civil,14.0,610.935
31550,278.00,RR,Relações Internacionais,1.0,518.540
23507,327.14,SP,Letras - Inglês,1.0,530.340
22114,347.49,PR,Pedagogia,1.0,496.020
41232,149.00,MG,Gestão Comercial,1.0,571.200
32123,277.00,RS,Serviço Social,1.0,577.680
35767,257.19,SP,Letras - Português e Espanhol,1.0,546.480
31947,278.00,SP,Gestão Ambiental,1.0,450.000
27911,298.00,MG,Ciências Contábeis,1.0,555.740


In [77]:
addprouni = pd.read_json("pure_data/prouni.json")

adicionalprouni = pd.DataFrame({
    'curso': addprouni['NOME_CURSO_BOLSA'],
    'estado': addprouni['SIGLA_UF_BENEFICIARIO_BOLSA'],
    'sexo': addprouni['SEXO_BENEFICIARIO_BOLSA'].map(lambda sexo: 'Masculino' if sexo == 'M' else 'Feminino'),
    'raca': addprouni['RACA_BENEFICIARIO_BOLSA'].map(lambda raca: 'Outro' if raca not in ['Parda', 'Branca', 'Preta'] else raca),
    'deficiente': addprouni['BENEFICIARIO_DEFICIENTE_FISICO'].map(lambda deficiente: 'Sim' if deficiente == 'S' else 'Não'),
    'idade': addprouni['DT_NASCIMENTO_BENEFICIARIO'].map(procdata.get_age_from_birthdate)
})
#addprouni.head(3)
print(adicionalprouni.__len__(), "rows")
adicionalprouni.sample(n=10)

236636 rows


,curso,estado,sexo,raca,deficiente,idade
219829,Pedagogia,MA,Feminino,Parda,Não,25
233459,Engenharia De Produção,MA,Masculino,Parda,Não,24
200465,Enfermagem,MA,Masculino,Branca,Não,23
219591,Enfermagem,MA,Feminino,Branca,Não,35
180904,Curso Superior De Tecnologia Em Gastronomia,GO,Feminino,Branca,Não,24
219962,Ciências Contábeis,AC,Feminino,Parda,Não,30
187436,Pedagogia,MS,Feminino,Branca,Não,23
113714,Engenharia De Produção,RS,Masculino,Branca,Não,44
188936,Pedagogia,RJ,Feminino,Parda,Não,23
154159,Medicina Veterinária,SP,Feminino,Outro,Não,26


In [76]:
edbasica = pd.read_csv("pure_data/microdados_ed_basica_2021.csv", sep=";", encoding="latin-1")

educacaobasica = pd.DataFrame({
    'estado': edbasica['SG_UF'],
    'esgoto_inexistente': edbasica['IN_ESGOTO_INEXISTENTE'].map(procdata.get_label_from_floatbool),
    'energia_inexistente': edbasica['IN_ENERGIA_INEXISTENTE'].map(procdata.get_label_from_floatbool),
    'agua_inexistente': edbasica['IN_AGUA_INEXISTENTE'].map(procdata.get_label_from_floatbool),
    'acesso_internet': edbasica['IN_INTERNET'].map(procdata.get_label_from_floatbool),
    'faz_exame_selecao': edbasica['IN_EXAME_SELECAO'].map(procdata.get_label_from_floatbool),
    'especializada_deficientes': edbasica['IN_ESP'].map(procdata.get_label_from_floatbool),
    'ensino_tecnico': edbasica['IN_PROF_TEC'].map(procdata.get_label_from_floatbool)
})
#dfedbasica = spark.createDataFrame(geralprouni)
#edbasica.head(10)
print(educacaobasica.__len__(), "rows")
educacaobasica.sample(10)

/var/folders/90/4r9_jk_s59n2616g7y9snmn00000gn/T/ipykernel_27797/3757209670.py:1: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  edbasica = pd.read_csv("pure_data/microdados_ed_basica_2021.csv", sep=";", encoding="latin-1")


221140 rows


,estado,esgoto_inexistente,energia_inexistente,agua_inexistente,acesso_internet,faz_exame_selecao,especializada_deficientes,ensino_tecnico
208151,RS,Não,Não,Não,Sim,Não,Não,Não
55220,CE,Não,Não,Não,Sim,Não,Sim,Não
167832,SP,Não,Não,Não,Sim,Não,Sim,Não
200114,RS,Não,Não,Não,Sim,Não,Sim,Não
165015,SP,Não,Não,Não,Sim,Não,Não,Não
88265,BA,Não,Não,Não,Sim,Não,Sim,Não
201356,RS,Não,Não,Não,Sim,Não,Não,Não
41112,PI,Não,Não,Não,Não,Não,Sim,Não
36343,MA,Não,Não,Não,Não,Não,Não,Não
160993,SP,Não,Não,Não,Não,Não,Não,Não


In [79]:
# prouni
# adicionalprouni
# educacaobasica

aditional = []
educacaoaditional = []

#pd.Series(np.arange(prouni.__len__())).map(lambda _: )
arr = pd.DataFrame(np.zeros((prouni.__len__(), prouni.columns.__len__())))
#print(arr[1].map(lambda x: x+1))

df = prouni.join(adicionalprouni.set_index('estado'), on='estado')
print(df.head(10))
#spark.createDataFrame(prouni).rdd\
#    .map(lambda c: adicionalprouni[(adicionalprouni['estado']==c['estado'])].sample()['sexo'])\
#    .foreach(lambda x: print(x))

#rdd2 = rdd.map(lambda c: )
#rdd2.collect()
#df = spark.createDataFrame(prouni)
#df.
#df['estado'].map(lambda curso: adicionalprouni[(adicionalprouni['estado']==curso)].sample()['sexo'])
#df = prouni.copy()
#df['sexo'] = prouni['estado'].map(lambda curso: adicionalprouni[(adicionalprouni['estado']==curso)].sample()['sexo'])
#print(df['sexo'])
"""
prouni.reset_index()
for row in prouni.itertuples():

    prouni_adicional = adicionalprouni[(adicionalprouni['curso']==row.curso) & (adicionalprouni['estado']==row.estado)]
    educacao_adicional = educacaobasica[(educacaobasica['estado']==row.estado)]

    aditional.append(prouni_adicional.sample() if len(prouni_adicional) > 0 else adicionalprouni.sample())
    educacaoaditional.append(educacao_adicional.sample() if len(educacao_adicional) > 0 else educacaobasica.sample())

aditional
educacaoaditional
#pd.DataFrame(educacaoaditional)

newdf = pd.concat(aditional)
df2 = prouni.assign(ad = pd.Series(newdf['sexo']))
df2
newdf['sexo']
"""

KeyboardInterrupt: 

In [ ]:
from datetime import datetime
# dateparsed = datetime.strptime(date[:-5], "%Y-%m-%dT%H:%M:%S")



In [ ]:
addprouni = pd.read_json("pure_data/prouni.json")

adicionalprouni = pd.DataFrame({
    'curso': addprouni['NOME_CURSO_BOLSA'],
    'estado': addprouni['SIGLA_UF_BENEFICIARIO_BOLSA'],
    'sexo': addprouni['SEXO_BENEFICIARIO_BOLSA'].map(lambda sexo: 'Masculino' if sexo == 'M' else 'Feminino'),
    'raca': addprouni['RACA_BENEFICIARIO_BOLSA'].map(lambda raca: 'Outro' if raca not in ['Parda', 'Branca', 'Preta'] else raca),
    'deficiente': addprouni['BENEFICIARIO_DEFICIENTE_FISICO'].map(lambda deficiente: 'Sim' if deficiente == 'S' else 'Não'),
    'idade': addprouni['DT_NASCIMENTO_BENEFICIARIO'].map(procdata.get_age_from_birthdate)
})
#addprouni.head(3)
adicionalprouni.sample(n=10)

In [100]:
edbasica = pd.read_csv("pure_data/microdados_ed_basica_2021.csv", sep=";", encoding="latin-1")

educacaobasica = pd.DataFrame({
    'estado': edbasica['SG_UF'],
    'esgoto_inexistente': edbasica['IN_ESGOTO_INEXISTENTE'].map(procdata.get_label_from_floatbool),
    'energia_inexistente': edbasica['IN_ENERGIA_INEXISTENTE'].map(procdata.get_label_from_floatbool),
    'agua_inexistente': edbasica['IN_AGUA_INEXISTENTE'].map(procdata.get_label_from_floatbool),
    'acesso_internet': edbasica['IN_INTERNET'].map(procdata.get_label_from_floatbool),
    'faz_exame_selecao': edbasica['IN_EXAME_SELECAO'].map(procdata.get_label_from_floatbool),
    'especializada_deficientes': edbasica['IN_ESP'].map(procdata.get_label_from_floatbool),
    'ensino_tecnico': edbasica['IN_PROF_TEC'].map(procdata.get_label_from_floatbool)
})
#dfedbasica = spark.createDataFrame(geralprouni)
#edbasica.head(10)
educacaobasica.sample(10)

/tmp/ipykernel_12442/693207899.py:1: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  edbasica = pd.read_csv("pure_data/microdados_ed_basica_2021.csv", sep=";", encoding="latin-1")


,estado,esgoto_inexistente,energia_inexistente,agua_inexistente,acesso_internet,faz_exame_selecao,especializada_deficientes,ensino_tecnico
162696,SP,Não,Não,Não,Sim,Não,Sim,Não
14293,PA,Não,Não,Não,Sim,Não,Sim,Não
189361,PR,Não,Não,Não,Sim,Não,Sim,Não
129037,MG,Não,Não,Não,Não,Não,Não,Não
56514,CE,Não,Não,Não,Não,Não,Não,Não
112159,MG,Não,Não,Não,Sim,Não,Não,Não
19289,PA,Sim,Sim,Sim,Não,Não,Sim,Não
59350,RN,Não,Não,Não,Não,Não,Não,Não
203715,RS,Não,Não,Não,Sim,Não,Sim,Não
93812,BA,Não,Não,Não,Sim,Sim,Sim,Não


In [ ]:
adicionalprouni[(adicionalprouni['curso']=='Enfermagem') & (adicionalprouni['estado']=='SP')].sample(n=10)

In [14]:
from datetime import datetime
# dateparsed = datetime.strptime(date[:-5], "%Y-%m-%dT%H:%M:%S")

from difflib import SequenceMatcher
sm = SequenceMatcher(None, "S@o Pa*lo", "São Paulo")
sm.ratio()

0.7777777777777778